In [1]:
import os,glob
import numpy as np
import pandas as pd
import glob
import pyslha
import time
import progressbar as P
from atlas_susy_2018_13_Recast import getLLPs, getDisplacedJets, getJets
import matplotlib.pyplot as plt
import os,glob
import numpy as np
import seaborn as sns
from scipy.special import kn,zetac
import pyslha
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
sns.set_palette(sns.color_palette("Paired"))

delphesDir = os.path.abspath("./DelphesLLP")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT

ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')


Welcome to JupyROOT 6.30/02


True

In [2]:
inputFile = './pp2GoGo/Events/run_07/gluino_tau_1_delphes_events.root'

In [3]:
f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")
nevts = tree.GetEntries()

wHighPT = 0.0
wTrackless = 0.0
for ievt in range(nevts):    

    tree.GetEntry(ievt)
    llps = getLLPs(tree.llps,tree.llpDaughters)
    jets = getJets(tree.GenJet,pTmin=25.,etaMax=5.0)
    jetsDisp = getDisplacedJets(jets,llps)
    
    # Event acceptance:
    highPT_acc = acceptance(jets,jetsDisp,sr='HighPT')
    trackless_acc = acceptance(jets,jetsDisp,sr='Trackless')                        
    if (not highPT_acc) and (not trackless_acc):
        continue
    
    # Event efficiency
    highPT_eff = eventEff(jets,llpCandidates,sr='HighPT')
    trackless_eff = eventEff(jets,llpCandidates,sr='Trackless')
    
    # Vertex acceptances:
    v_acc = np.array([vertexAcc(llp) for llp in llpCandidates])
    
    # Vertex efficiencies:
    v_eff = np.array([vertexEff(llp) for llp in llpCandidates])
    
    wvertex = 1.0-np.prod(1.0-v_acc*v_eff)
    
    # Add to the total weight in each SR:
    wHighPT += weight*highPT_acc*highPT_eff*wvertex
    wTrackless += weight*highPT_acc*highPT_eff*wvertex
    
    
return {'highPT (pb)' : wHighPT, 'trackless (pb)' : wTrackless}
    
